https://stats.stackexchange.com/questions/134282/relationship-between-svd-and-pca-how-to-use-svd-to-perform-pca
https://mccormickml.com/2014/06/03/deep-learning-tutorial-pca-and-whitening/
http://ufldl.stanford.edu/tutorial/unsupervised/PCAWhitening/
https://www.cs.bgu.ac.il/~inabd171/wiki.files/lecture14_handouts.pdf
http://www.stat.columbia.edu/~fwood/Teaching/w4315/Fall2009/pca.pdf
http://www.math.uwaterloo.ca/~aghodsib/courses/f06stat890/notes/lec6.pdf

Cas de la PCA et de la SVD ? Matrice des données nxp : nxn x nxp x pxp 

Quelle que soit l'approche du problème de la PCA, les données sont toujours centrées: $\frac{1}{n}\sum_{i}x_{i} = 0$. Celà simplifie les calculs, la matrice de covariance étant alors notamment égale à $XX^{\top}$.

## Approche géométrique par la variance
On cherche les directions de variance maximale ou de façon équivalente, d'inertie minimale.

## Normalisation 

### Centrage
La PCA travaille sur des données centrées (moyenne nulle). Au delà du fait que supposer les données centrées simplifie les calculs et permet de donner d'améliorer l'interprétabilité de la méthode (le centrage fait qu'elle revient à diagonaliser la matrice de covariance), la PCA s'interprète aussi comme une rotation des axes du repère pour obtenir les directions de variance maximale. Il est alors sous entendu que le repère est alors centré sur le nuage dont on recherche les directions de variance maximale. Réaliser la décomposition de la matrice de covariance sur des données non centrées risque de d'aboutir à des résultats différents. 

On précise que `sklearn` centre toujours la matrice des données `X` avant de procéder à la SVD que celle-ci ait été centrée au préalable ou non. On est donc garanti que les composantes retournées par `sklearn.decomposition.PCA` par exemple sont toujours calculées sur des données centrées.

### Réduction
La PCA n'exige pas que les données aient été normalisées. Une PCA réalisée sur des données centrées revient à diagonaliser la matrice de covariance alors qu'une PCA réalisée sur des données centrées-réduites revient à diagonaliser la matrice de corrélation.

A la réduction est associée une perte d'information: celle des variance relatives entre les variables. Les PCA réalisées sur la matrices de covariance et de corrélation donneront des résultats différents. Normaliser les données est cependant parfois nécessaire dans les cas où les plages de valeurs prises par les données sont très différentes. La PCA cherchant les directions de variance maximale (et donc travaillant sur des carrés d'écarts), si une variable se place sur une échelle très différente de celle des autres, elle risque d'avoir une contribution disproportionnée à la variance totale. Les résultats obtenus seront alors biaisés en faveur des variables se distribuant sur les plages de valeurs les plus élevées. Si toutes nos variables se distribuent sur des valeurs comparables, on peut ne pas normaliser, les informations extraites par la PCA n'en seront que plus riches. 

L'attitude à adopter vis-à-vis de la réduction dans le cas de la PCA est donc une position de compromis.

Bien que `sklearn` centre toujours les données avant de procéder à la SVD, rien n'est fait pour le *scaling* des données dont la responsabilité appartient à l'utilisateur. 

# Distance de Mahalanobis
Soit $M$ un ensemble de points de $\mathbb{R}^p$ (espace des *features* de dimension $p$) de matrice de covariance $\Sigma$ et de moyenne $\mu$ et $x$ un point de $\mathbb{R}^p$, on définit la distance de Mahalanobis de $x$ à la distribution $M$ comme:

$$D(x,M)=\sqrt{(x-\mu)^{\top}\Sigma^{-1}(x-\mu)}$$

Comment interpréter cette distance ? Prenons le cas particulier de $\Sigma$ diagonale et supposons $M$ composé de $n$ réalisations d'une gaussienne multivariée $\mathcal{N}(\mu, \Sigma)$. On remarque la distance de Mahalanobis prend alors la forme d'une distance euclidienne où chaque dimension est pondérée par l'écart-type de $M$ dans cette direction: 

$$D(x,M)=\sqrt{\sum_{i}\frac{(x_{i}-\mu_{i})^2}{\sigma_{i}}}$$

Tout se passe comme s'il y avait normalisation avant calcul de la distance. Il s'agit alors d'un calcul de distance euclidienne mais dans un espace transformé où chacune des directions a été dilatée/contractée en fonction des propriétés de la distribution $M$. La distance de Mahalanobis correspond en fait à un calcul de distance dans le système d'axes (de variance maximale) de la distribution $M$, que ce passe-t-il quand il y a de la covariance, i.e. que les axes de $M$ ne coïncident plus avec les axes canoniques? C'est le cas le plus général expliquant la multiplication par $\Sigma^{-1}$.

Si les axes ne coïncident pas avec les directions de variance de $M$, on va simplement pivoter le repère de façon à ce que ce soit le cas. On va donc en fait réaliser une PCA, dans cette nouvelle base, $\Sigma$ est diagonale, on a ainsi: $\Sigma=P^{\top}\Delta P$ avec $P$ la matrice de changement de base. La distance de Mahalanobis s'écrit alors: 

$$D(x,M)=\sqrt{(x-\mu)^{\top}\Sigma^{-1}(x-\mu)} = \sqrt{(x-\mu)^{\top}P^{\top}\Delta^{-1}P(x-\mu)} = \sqrt{y^{\top}y}$$

Avec $y=\Delta^{-1/2}P(x-\mu)$ Cette nouvelle forme a deux conséquences:
* Elle nous permet de donner une interprétation générale à la distance de Mahalanobis: on peut la voir comme un calcul de distance euclidienne précédée de plusieurs traitements: 1) On se place dans le système d'axes de la distribution $M$ (multplication par $P$) 2) On dilate/contracte l'espace différemment suivant chaque direction de façon inversement proportionnelle à la valeur propre pour cette direction (multplication par $\Delta^{-1/2}$, étape aussi appelée *whitening*) 3) Dans ce dernier espace où $M$ est centrée réduite, on procède à un simple calcul de distance euclidienne.
* La dernière égalité ci-dessus permet aussi de montrer que sous l'hypothèse de $M$ suivant une $\mathcal{N}(\mu, \Sigma)$, $D(x,M)^2$ suit un chi-deux à $p$ degrés de liberté. On peut ainsi associer à valeur de distance de Mahalanobis une p-value correspondant à la problabilité de trouver une distance plus élevée sous l'hypothèse précédente. Ainsi, de grandes valeurs de distance seront associées à de faibles p-values. Le passage aux p-values a aussi l'avantage de *mapper* les distances dans l'intervalle $[0,1]$ (normalisation).

On remarque que l'expression de la distance de Mahalanobis est proche de celle de la distance euclidienne dans $\mathbb{R}^p$, les deux coïncident en effet quand $\Sigma^{-1}=I_p$.

Il est aussi possible de calculer la distance de Mahalanobis entre deux vecteurs $x$ et $y$ de $\mathbb{R}^p$, l'expression de modifie simplement en:

$$D(x,y)=\sqrt{(x-y)^{\top}\Sigma^{-1}(x-y)}$$

La distance s'interprète alors comme la distance entre deux réalisations d'une même variable aléatoire multivariée de matrice de covariance $\Sigma$. Elle vient quantifier la dissimilarité entre deux vecteurs $x$ et $y$ dont on sait ou dont on fait l'hypothèse qu'ils sont issus de la même distribution.

On peut citer comme désavantage majeur de la distance de Mahalanobis la nécéssité d'estimer et d'inverser $\Sigma$ (en supposant que celà soit possible).

Pourquoi?
    Naivement le calcul d'un point à une distribution utilisant la distance euclidienne va simplement calculer la distance au barycentre et risque d'être aveugle à un certain nombre de choses. La distance de Mahalanobis permet de calculer une distance (euclidienne) dans un repère / après déformation ad hoc (le système d'axes correspondant à ses directions de variance maximale) propre aux spécificités de la distribution de l'ensemble $M$.
    
On s'en sert pour détecter des outliers.